In [8]:
# Importing spark and dependencies

import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.context import SparkContext

In [2]:
# Pointing to our Google Project credentials

credentials_location = '/home/abhirup.ghosh/.gc/gcp_service_capstone.json'

In [3]:
# Setting up spark cluster configurations

conf = SparkConf() \
    .setMaster('local[*]') \
    .setAppName('test') \
    .set("spark.jars", "./lib/gcs-connector-hadoop3-2.2.5.jar") \
    .set("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", credentials_location)

sc = SparkContext(conf=conf)

hadoop_conf = sc._jsc.hadoopConfiguration()

hadoop_conf.set("fs.AbstractFileSystem.gs.impl",  "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hadoop_conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_conf.set("fs.gs.auth.service.account.json.keyfile", credentials_location)
hadoop_conf.set("fs.gs.auth.service.account.enable", "true")

24/03/30 21:30:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
# Setting up spark cluster

spark = SparkSession.builder \
    .config(conf=sc.getConf()) \
    .getOrCreate()

In [38]:
# reading data as spark dataframes with previously 

df_appearances = spark.read\
                    .parquet('gs://capstone_datalake/raw/appearances.parquet')

df_competitions = spark.read\
                    .parquet('gs://capstone_datalake/raw/competitions.parquet')

In [49]:
# merging the data to form one dataset

df_data_all = df_appearances.join(df_competitions, on='competition_id', how='left')

In [50]:
df_data_all.registerTempTable('data_all')

/home/abhirup.ghosh/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [52]:
spark.sql("""
SELECT *
FROM
    data_all
LIMIT 10
""").show()

+--------------+--------------+-------+---------+--------------+----------------------+-------------------+----------------+------------+---------+-----+-------+--------------+--------------------+--------------------+--------------------+-----------------+----------+------------+--------------------+-------------+--------------------+------------------------+
|competition_id| appearance_id|game_id|player_id|player_club_id|player_current_club_id|               date|     player_name|yellow_cards|red_cards|goals|assists|minutes_played|    competition_code|                name|            sub_type|             type|country_id|country_name|domestic_league_code|confederation|                 url|is_major_national_league|
+--------------+--------------+-------+---------+--------------+----------------------+-------------------+----------------+------------+---------+-----+-------+--------------+--------------------+--------------------+--------------------+-----------------+----------+------

In [53]:
df_data_all.coalesce(1).write.parquet('data_all.parquet', mode='overwrite')